In [2]:
#!/usr/bin/env python
# coding: utf-8
import sys
import random
import math
import numpy as np
from operator import itemgetter
from collections import defaultdict

# 使得随机数据可预测，即只要seed的值一样，后续生成的随机数都一样
random.seed(0)
# 创建UserCF对象
class UserCF(object):
    def __init__(self):
        self.trainset = {}
        self.testset = {}
        self.n_sim_user = 1000  # n_sim_user: top n个用户
        self.n_rec_item = 10  # n_rec_item: top n个推荐结果
        self.user_sim_mat = {}  # user_sim_mat: 用户之间的相似度
        self.item_popular = {}  # item_popular: 训练集商品被购买的次数
        self.item_count = 0  # item_count:训练集 总商品数量
        self.trainset_len = 0 #训练集合样本总数
        self.testset_len = 0#测试集合样本总数
        print ('相似的用户数量 = %d' % self.n_sim_user, file=sys.stderr)
        print ('推荐的商品数量 = %d' % self.n_rec_item, file=sys.stderr)
    @staticmethod
    def loadfile(filename):#打开文件，确定字段
        ''' load a file, return a generator. '''
        fp = open(filename, 'r')
        for i, line in enumerate(fp):
            if i == 0: 
                continue
            yield line.strip('\r\n')
            if i % 100000 == 0:
                print ('loading %s(%s)' % (filename, i), file=sys.stderr)
        fp.close()

    def generate_dataset(self, filename, pivot=0.8):#'拆分训练集和测试集'

        for line in self.loadfile(filename):
            ID, user, item, num, rating, price = line.split(',')
            if random.random() <= pivot:#比例
                self.trainset.setdefault(user, {})
                self.trainset[user][item] = float(rating)
                self.trainset_len += 1
            else:
                self.testset.setdefault(user, {})
                self.testset[user][item] = float(rating)
                self.testset_len += 1
        print ('训练集 = %s' % self.trainset_len, file=sys.stderr)
        print ('测试集 = %s' % self.testset_len, file=sys.stderr)

    def calc_user_sim(self):# 计算用户之间的相似度
        ''' calculate user similarity matrix '''
        # build inverse table for item-users（key=itemID, value=list of userIDs who have seen this item）
#         print ('=======================================', file=sys.stderr)
        item2users = dict()#构建商品到用户的字典
#     商品流行度：商品被多少用户购买；
        for user, items in self.trainset.items():
            for item in items:
                # inverse table for item-users
                if item not in item2users:
                    item2users[item] = set()
                item2users[item].add(user)
                # count item popularity at the same time
                if item not in self.item_popular:
                    self.item_popular[item] = 0
                self.item_popular[item] += 1
#         print ('build item-users inverse table succ', file=sys.stderr)
        # save the total item number, which will be used in evaluation
        self.item_count = len(item2users)
        print ('总商品数量 = %d' % self.item_count, file=sys.stderr)
        # count co-rated items between users
        usersim_mat = self.user_sim_mat
#         print ('building user co-rated items matrix...', file=sys.stderr)
        for item, users in item2users.items():
            for u in users:
                usersim_mat.setdefault(u, defaultdict(int))
                for v in users:
                    if u == v:
                        continue
                    usersim_mat[u][v] += 1
#         print ('build user co-rated items matrix succ', file=sys.stderr)
        # calculate similarity matrix
#         print ('calculating user similarity matrix...', file=sys.stderr)
        simfactor_count = 0
        PRINT_STEP = 2000000
        for u, related_users in usersim_mat.items():
            for v, count in related_users.items():
                usersim_mat[u][v] = count / math.sqrt(
                    len(self.trainset[u]) * len(self.trainset[v]))
                simfactor_count += 1
#         print ('calculate user similarity matrix(similarity factor) succ',file=sys.stderr)
        print ('总相似因子数 = %d' %
               simfactor_count, file=sys.stderr)
    def recommend(self, user):
        ''' Find K similar users and recommend N items '''
        K = self.n_sim_user#K为相似用户数量
        N = self.n_rec_item#N为推荐商品数量
        rank = dict()#构建rank字典
        watched_items = self.trainset[user]
        for similar_user, similarity_factor in sorted(self.user_sim_mat[user].items(),
                                                      key=itemgetter(1), reverse=True)[0:K]:
            for item in self.trainset[similar_user]:
                if item in watched_items:
                    continue
                # predict the user's "interest" for each item
                rank.setdefault(item, 0)
                rank[item] += similarity_factor
        # return the N best items
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[0:N]
    
    # 评估推荐效果
    def evaluate(self,k):
        hit = 0
        rec_count = 0
        test_count = 0
        all_rec_items = set()
        popular_sum = 0
        MRR_List = []
        for i, user in enumerate(self.trainset):
            test_items = self.testset.get(user, {})#测试集用户的购买列表
            rec_items = self.recommend(user)  #训练集合给用户的推荐列表
            rec_items=rec_items[0:k]
            #print('user=',user)
            #print('Top-'+str(k)+' 推荐列表',rec_items[0:k])
            #print('==============================================')
            Index = 0  # 排序
            for item, _ in rec_items:
                Index += 1
                if item in test_items:
                    hit += 1
                    MRR_List.append(1/float(Index))
                all_rec_items.add(item)
                popular_sum += math.log(1 + self.item_popular[item])
            rec_count += k
            test_count += len(test_items)
        precision = hit / (1.0 * rec_count)
        recall = hit / (1.0 * test_count)
        F_score=2*precision*recall/(precision+recall)
        hitrate = hit / (self.testset_len)
        coverage = len(all_rec_items) / (1.0 * self.item_count)
        #print('MRR_List:',MRR_List)
        MRR = round(np.mean(MRR_List), 4)
        popularity = popular_sum / (1.0 * rec_count)
        #print('Top' + str(k) + '推荐性能：')
        print('Top' + str(k) + '推荐性能：',
              'precision=%.4f\trecall=%.4f\tF_score=%.4f\tHitRate=%.4f\tcoverage=%.4f\tMRR=%.4f\tpopularity=%.4f' % (
              precision, recall, F_score, hitrate, coverage, MRR, popularity), file=sys.stderr)

if __name__ == '__main__':
    print('UCF推荐：')
    usercf = UserCF()
    ratingfile = 'userid-itemid-num-rating.csv'
    usercf.generate_dataset(ratingfile)
    usercf.calc_user_sim()
    usercf.evaluate(3)
    usercf.evaluate(5)
    usercf.evaluate(10)

UCF推荐：


相似的用户数量 = 1000
推荐的商品数量 = 10
训练集 = 34910
测试集 = 8670
总商品数量 = 162
总相似因子数 = 63361416
Top3推荐性能： precision=0.0257	recall=0.5757	F_score=0.0493	HitRate=0.2355	coverage=0.8086	MRR=0.7763	popularity=4.4695
Top5推荐性能： precision=0.0180	recall=0.6699	F_score=0.0350	HitRate=0.2740	coverage=0.8642	MRR=0.6995	popularity=4.3247
Top10推荐性能： precision=0.0103	recall=0.7649	F_score=0.0203	HitRate=0.3129	coverage=0.9321	MRR=0.6296	popularity=3.9342


In [1]:
#!/usr/bin/env python
# coding: utf-8
# Item Based Collaborative Filtering
import sys
import random
import math
import os
import time
import numpy as np
from operator import itemgetter
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
#使得随机数据可预测，即只要seed的值一样，后续生成的随机数都一样
random.seed(0)
class ItemCF(object):
    def __init__(self):
        self.trainset = {}
        self.testset = {}
        self.n_sim_item = 40  # n_sim_item: top n个商品
        self.n_rec_item = 10 # n_rec_item: top n个推荐结果
        self.item_sim_mat = {}  # item_sim_mat: 商品之间的相似度
        self.item_popular = {}  # item_popular: 训练集商品被购买的次数
        self.item_count = 0  # item_count: 训练集总商品数量
        self.trainset_len = 0 #训练集合样本总数
        self.testset_len = 0#测试集合样本总数
        print('相似商品数量 = %d' % self.n_sim_item, file=sys.stderr)  # sys.stderr 目的就是返回错误信息
        print('推荐商品数量 = %d' % self.n_rec_item, file=sys.stderr)
    @staticmethod
    def loadfile(filename):
        fp = open(filename, 'r')
        # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
        for i, line in enumerate(fp):
            if i == 0:  
                continue
            yield line.strip('\r\n')
            #print (i,line)
        fp.close()

    def generate_dataset(self, filename, pivot=0.8):#'拆分训练集和测试集'

        for line in self.loadfile(filename):
            ID, user, item, num, rating, price = line.split(',')
            if random.random() <= pivot:#比例
                self.trainset.setdefault(user, {})
                self.trainset[user][item] = float(rating)
                self.trainset_len += 1
            else:
                self.testset.setdefault(user, {})
                self.testset[user][item] = float(rating)
                self.testset_len += 1
        print ('训练集 = %s' % self.trainset_len, file=sys.stderr)
        print ('测试集 = %s' % self.testset_len, file=sys.stderr)

    def calc_item_sim(self):#       计算商品相似矩阵
        #print('计算商品数量和受欢迎程度...', file=sys.stderr)
#         print('self.trainset.items()=',item)
        for user, items in self.trainset.items():  # dict.items() 以列表返回可遍历的(键, 值) 元组数组
            for item in items: # 计算商品受欢迎程度
                if item not in self.item_popular:
                    self.item_popular[item] = 0
                self.item_popular[item] += 1
#         print('计算商品数量和受欢迎程度', file=sys.stderr)
        self.item_count = len(self.item_popular)# 保存总商品数量
        #print('总商品数量 = %d' % self.item_count, file=sys.stderr)
        itemsim_mat = self.item_sim_mat# count co-rated users between items
        for user, items in self.trainset.items():#print('建立共同评级用户矩阵...', file=sys.stderr)
            for m1 in items:
                itemsim_mat.setdefault(m1, defaultdict(int))
                for m2 in items:
                    if m1 == m2:
                        continue
                    itemsim_mat[m1][m2] += 1  #python的二维字典
#        print('建立共同评级用户矩阵成功', file=sys.stderr)
        #**********计算商品相似矩阵**********#
        simfactor_count = 0
        PRINT_STEP = 2000000
        for m1, related_items in itemsim_mat.items():
            for m2, count in related_items.items():
                itemsim_mat[m1][m2] = count / math.sqrt(
                    self.item_popular[m1] * self.item_popular[m2])
                simfactor_count += 1
        #print('总相似因子数 = %d' %simfactor_count, file=sys.stderr)
        return self.item_popular
    def recommend(self, user):#Top-N推荐
        K = self.n_sim_item
        N = self.n_rec_item
        rank = {}
        watched_items = self.trainset[user]#用户访问的商品
        for item, rating in watched_items.items():
            # itemgetter函数用于获取对象的哪些维的数据
            #python 里面的字典
#             itemgetter（1）是获取key部分的sorted(words.iteritems(), key=itemgetter(1), reverse=True)[:N] 是按key大小排列字典
            for related_item, similarity_factor in sorted(self.item_sim_mat[item].items(),
                                                           key=itemgetter(1), reverse=True)[:K]:
                if related_item in watched_items:
                    continue
                rank.setdefault(related_item, 0)
                rank[related_item] += similarity_factor * rating
#                 print('rating=',rating)
#                 print('similarity_factor=',similarity_factor)
#                 print('rank[related_item]=',rank[related_item])
        return sorted(rank.items(), key=itemgetter(1), reverse=True)[:N]# 返回N个商品

    def evaluate(self,k):
        hit = 0 
        rec_count = 0 
        test_count = 0
        all_rec_items = set()
        popular_sum = 0
        MRR_List = []
        for i, user in enumerate(self.trainset):
            test_items = self.testset.get(user, {})#测试集用户的购买列表
            #print('test_items',test_items)
            rec_items = self.recommend(user)  #训练集合给用户的推荐列表
            rec_items=rec_items[0:k]
            #print('user=',user)
            #print('Top-'+str(k)+' 推荐列表',rec_items[0:k])
            #print('==============================================')
            Index = 0  # 排序
            for item, _ in rec_items:
                Index += 1
                if item in test_items:
                    hit += 1
                    MRR_List.append(1/float(Index))
                all_rec_items.add(item)
                popular_sum += math.log(1 + self.item_popular[item])
            rec_count += k
            test_count += len(test_items)
        precision = hit / (1.0 * rec_count)
        recall = hit / (1.0 * test_count)
        #print('test_count:',test_count)
        F_score=2*precision*recall/(precision+recall)
        hitrate=hit / (self.testset_len)
        #print('testset_len:',self.testset_len)
        coverage = len(all_rec_items) / (1.0 * self.item_count)

        #print('MRR_List:',MRR_List)
        MRR = round(np.mean(MRR_List), 4)
        popularity = popular_sum / (1.0 * rec_count)
        #print('Top' + str(k) + '推荐性能：')
        print ('Top' + str(k) + '推荐性能：','precision=%.4f\trecall=%.4f\tF_score=%.4f\tHitRate=%.4f\tcoverage=%.4f\tMRR=%.4f\tpopularity=%.4f' %(precision, recall,F_score,hitrate,coverage, MRR, popularity), file=sys.stderr)

if __name__ == '__main__': 
    print('ICF推荐：')
    itemcf = ItemCF() #实例化该类
    ratingfile =  'userid-itemid-num-rating.csv'
    itemcf.generate_dataset(ratingfile) #加载数据，划分数据
    itemcf.calc_item_sim()    #计算相似度
    itemcf.evaluate(3)  # Top-3评价结果
    itemcf.evaluate(5)  # Top-5评价结果
    itemcf.evaluate(10)   #Top-10评价结果






相似商品数量 = 40
推荐商品数量 = 10
训练集 = 34910
测试集 = 8670
Top3推荐性能： precision=0.0253	recall=0.5655	F_score=0.0484	HitRate=0.2314	coverage=0.9198	MRR=0.7866	popularity=5.9797
Top5推荐性能： precision=0.0180	recall=0.6704	F_score=0.0350	HitRate=0.2743	coverage=0.9321	MRR=0.6999	popularity=5.8035
Top10推荐性能： precision=0.0109	recall=0.8156	F_score=0.0216	HitRate=0.3337	coverage=0.9383	MRR=0.5990	popularity=5.5561
